In [1]:
pip install flaml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.3/313.3 kB 6.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
from flaml import autogen
from flaml import AutoML
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.metrics import accuracy_score


2024-10-20 14:09:46,384	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-10-20 14:09:48,174	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
import pandas as pd

In [4]:
df=pd.read_csv("/kaggle/input/encryption-dataset-2/encryption_attack_dataset.csv")

In [5]:
df

,Algorithm,Key Size,Attack Type,Attack Success
0,AES,128,Grover's Algorithm,True
1,McEliece,2048,Cryptoanalyzing,False
2,Kyber,192,Cryptoanalyzing,False
3,RC4,128,Shor's Algorithms,False
4,AES,128,Grover's Algorithm,False
...,...,...,...,...
19995,RC4,128,Cryptoanalyzing,True
19996,AES,128,Shor's Algorithms,True
19997,McEliece,2048,Shor's Algorithms,False
19998,Kyber,192,Shor's Algorithms,False


In [6]:
X=df[['Algorithm', "Key Size","Attack Type"]]
y=df["Attack Success"]

In [7]:
y

0         True
1        False
2        False
3        False
4        False
         ...  
19995     True
19996     True
19997    False
19998    False
19999     True
Name: Attack Success, Length: 20000, dtype: bool

In [8]:
X_train, X_test, y_train, y_test= train_test_split(X, y,test_size=0.2, stratify=y, random_state=66 )

In [9]:
X_train

,Algorithm,Key Size,Attack Type
5677,McEliece,2048,Brute Force
8867,RC4,128,Grover's Algorithm
2709,McEliece,2048,Cryptoanalyzing
19808,AES,128,Grover's Algorithm
13041,McEliece,2048,Cryptoanalyzing
...,...,...,...
2363,RC4,128,Grover's Algorithm
10993,McEliece,2048,Grover's Algorithm
303,RC4,128,Cryptoanalyzing
4715,RC4,128,Brute Force


In [10]:
automl = AutoML()

automl.fit(X_train, y_train, time_budget=120)  




[flaml.automl.logger: 10-20 14:10:00] {1728} INFO - task = classification
[flaml.automl.logger: 10-20 14:10:00] {1739} INFO - Evaluation method: cv
[flaml.automl.logger: 10-20 14:10:00] {1838} INFO - Minimizing error metric: 1-roc_auc
[flaml.automl.logger: 10-20 14:10:01] {1955} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'sgd', 'catboost', 'lrl1']
[flaml.automl.logger: 10-20 14:10:01] {2258} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 10-20 14:10:01] {2393} INFO - Estimated sufficient time budget=2364s. Estimated necessary time budget=58s.
[flaml.automl.logger: 10-20 14:10:01] {2442} INFO -  at 0.5s,	estimator lgbm's best error=0.1359,	best estimator lgbm's best error=0.1359
[flaml.automl.logger: 10-20 14:10:01] {2258} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 10-20 14:10:01] {2442} INFO -  at 0.7s,	estimator lgbm's best error=0.1359,	best estimator lgbm's best error=0.1359
[flaml.automl.lo

In [11]:
print("Best Model:", automl.best_estimator)
print("Best Loss on validation set:", automl.best_loss)

Best Model: extra_tree
Best Loss on validation set: 0.13272721636896895


In [12]:
test_accuracy = automl.score(X_test, y_test)
print("Test accuracy:", test_accuracy)

Test accuracy: 0.822


In [13]:
model=automl
filename = 'encryption_QPC_14.pkl'


In [14]:
with open(filename, 'wb') as file:
    pickle.dump(model, file)


In [15]:
with open(filename, 'rb') as file:
    loaded_model = pickle.load(file)

In [16]:

test_accuracy = loaded_model.score(X_test, y_test)


In [17]:
test_accuracy

0.822

In [18]:
le_algorithm = LabelEncoder()
le_attack_type = LabelEncoder()

# Apply label encoding to 'Algorithm' and 'Attack Type'
X['Algorithm'] = le_algorithm.fit_transform(X['Algorithm'])
X['Attack Type'] = le_attack_type.fit_transform(X['Attack Type'])

/tmp/ipykernel_17/2150996034.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Algorithm'] = le_algorithm.fit_transform(X['Algorithm'])
/tmp/ipykernel_17/2150996034.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Attack Type'] = le_attack_type.fit_transform(X['Attack Type'])


In [19]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an XGBoost model
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

In [20]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.82


In [21]:
filename = 'encryption_QPC_15_XG.pkl'


In [22]:
with open(filename, 'wb') as file:
    pickle.dump(model, file)
